In [1]:
import numpy as np
import math

# Probabilistic Neural Network with 4 layers
class PNN(object):
    def __init__(self):
        self.L2 = []         # Layer 2 that holds the patterns
        print('Empty PNN created.')

    def train(self, X, y, p=2):
        self.n_ = X.shape[1]  # num of features
        self.p_ = p           # num of classes
        
        # Layer 2 (Pattern): Set up empty lists for each class
        for k in range(self.p_):
            self.L2.append([])    # Using Python's basic lists because ndarray cannot append empty arrays
                                  # Also perhaps we might have to input different data types

        # Enter patterns into Layer 2
        for i in range(X.shape[0]):
            self.L2[y[i]].append(X[i])

        print('PNN with', self.p_, 'classes trained.')

    def crossValidate(self, X, y, sigma=0.5):
        result = self.predict(X, sigma)
        num_correct = sum(result[:, 0] == y)

        print('Cross validation accuracy with sigma', sigma, ':', num_correct/len(y) * 100, '%')

    def predict(self, X, sigma=0.5):
        self.sigma_ = sigma   # smoothing parameter, not standard deviation
        
        m = X.shape[0]
        accL3 = np.zeros((m, self.p_))
        accL4 = np.zeros(m)
       
        # Layer 1 (Input): x
        for i in range(m):
            x = X[i]

            # Layer 3 (Averaging): for each class
            self.L3_ = np.zeros(self.p_)
            for k in range(self.p_):
                for ki in range(len(self.L2[k])):
                    self.L3_[k] += self._activation(x, self.L2[k][ki])
                self.L3_[k] /= len(self.L2[k])
                
                # Multiply constant
                self.L3_[k] *= (math.sqrt(2*math.pi) * self.sigma_)**(- self.n_)
                accL3[i][k] = self.L3_[k]

            # Layer 4 (Output/Decision): Maxing
            self.L4_ = self.L3_.argmax()
            accL4[i] = self.L4_

        return np.column_stack((accL4, accL3))

    def _activation(self, x, w):
        diff = x - w
        return math.exp( - np.dot(diff, diff) / (2 * self.sigma_**2) )


# Normalize to unit length: [0, 1]
# X must be ndarray
def Normalize(X):
    x_max = X.max(axis=0)
    x_min = X.min(axis=0)
    return (X - x_min) / (x_max - x_min)

In [2]:
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)
df.tail()

,0,1,2,3,4
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica
149,5.9,3.0,5.1,1.8,Iris-virginica


In [3]:
X = df.iloc[:, :4].values

y = df.iloc[:, 4].values
y[:50] = 0
y[50:100] = 1
y[100:] = 2

X_N = Normalize(X)

# Training data
X_tr = np.row_stack((X_N[0:40], X_N[50:90], X_N[100:140]))
y_tr = np.append(np.append(y[0:40], y[50:90]), y[100:140])

# Cross validation data (may be same as training data for a PNN)
X_cv = X_tr
y_cv = y_tr

# Test data
X_tt = np.row_stack((X_N[40:50], X_N[90:100], X_N[140:150]))
y_tt = np.append(np.append(y[40:50], y[90:100]), y[140:150])

In [4]:
pnn = PNN()

Empty PNN created.


In [5]:
pnn.train(X_tr, y_tr, p=3)

PNN with 3 classes trained.


In [6]:
n_iter = 10
for s in range(n_iter):
    pnn.crossValidate(X_cv, y_cv, sigma=(s+1)/n_iter)

Cross validation accuracy with sigma 0.1 : 97.5 %
Cross validation accuracy with sigma 0.2 : 95.0 %
Cross validation accuracy with sigma 0.3 : 93.3333333333 %
Cross validation accuracy with sigma 0.4 : 90.8333333333 %
Cross validation accuracy with sigma 0.5 : 90.0 %
Cross validation accuracy with sigma 0.6 : 90.0 %
Cross validation accuracy with sigma 0.7 : 90.0 %
Cross validation accuracy with sigma 0.8 : 90.0 %
Cross validation accuracy with sigma 0.9 : 90.0 %
Cross validation accuracy with sigma 1.0 : 89.1666666667 %


In [8]:
pnn.predict(X_tt, sigma=0.2)

array([[  0.00000000e+00,   1.07741166e+01,   1.56835020e-02,
          4.91822629e-05],
       [  0.00000000e+00,   1.62208403e+00,   6.91375935e-02,
          1.39710138e-04],
       [  0.00000000e+00,   8.33815293e+00,   1.22895170e-02,
          2.76397380e-05],
       [  0.00000000e+00,   8.28425633e+00,   6.75636278e-02,
          5.39788341e-04],
       [  0.00000000e+00,   7.88460127e+00,   1.98765964e-02,
          1.37981866e-04],
       [  0.00000000e+00,   8.88399796e+00,   6.17833659e-02,
          1.75062129e-04],
       [  0.00000000e+00,   8.71468784e+00,   5.64125404e-03,
          2.01686345e-05],
       [  0.00000000e+00,   9.71942835e+00,   2.00969818e-02,
          4.69501372e-05],
       [  0.00000000e+00,   9.35852129e+00,   8.82996397e-03,
          3.15970629e-05],
       [  0.00000000e+00,   1.11170618e+01,   2.46684018e-02,
          6.10155854e-05],
       [  1.00000000e+00,   1.99285737e-02,   7.19971851e+00,
          1.50876871e+00],
       [  1.00000000e